d-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px; height: 163px">
</div>

# CSE Coding Assignment
## Instructions

- Please answer all questions
- You can use any language you wish (e.g. Python, Scala, SQL...)
- Several Markdown cells require completion. Please edit the Markdown cells to include your answer.
- Your final notebook should compile without errors when you click "Run All"

**Please do not publish questions. This is a confidential assignment.**

### Creating a Cluster

You will need to create a Databricks Cluster. More information on this process is available here: https://docs.databricks.com/user-guide/clusters/create.html

## Getting Started

**REQUIRED:** Run the following cells exactly as written to retrieve the necessary Coding Assignment Data Sets from Amazon S3.

In [0]:
%sh curl --remote-name-all 'https://files.training.databricks.com/assessments/cse-take-home/{covertype,kafka,treecover,u.data,u.item}.csv'


[1/5]: https://files.training.databricks.com/assessments/cse-take-home/covertype.csv --> covertype.csv
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   133  100   133    0     0   1242      0 --:--:-- --:--:-- --:--:--  1254

[2/5]: https://files.training.databricks.com/assessments/cse-take-home/kafka.csv --> kafka.csv
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  135k  100  135k    0     0  1717k      0 --:--:-- --:--:-- --:--:-- 1717k

[3/5]: https://files.training.databricks.com/assessments/cse-take-home/treecover.csv --> treecover.csv
  % Total    % Received % Xferd  Average Speed   Time    Time 

In [0]:
dbutils.fs.cp("file:/databricks/driver/covertype.csv", "dbfs:/FileStore/tmp/covertype.csv")
dbutils.fs.cp("file:/databricks/driver/kafka.csv", "dbfs:/FileStore/tmp/kafka.csv")
dbutils.fs.cp("file:/databricks/driver/treecover.csv", "dbfs:/FileStore/tmp/treecover.csv")
dbutils.fs.cp("file:/databricks/driver/u.data.csv", "dbfs:/FileStore/tmp/u.data.csv")
dbutils.fs.cp("file:/databricks/driver/u.item.csv", "dbfs:/FileStore/tmp/u.item.csv")

Out[111]: True

## Part 1: Reading and Parsing Data

### Question 1:  Code Challenge - Load a CSV

- Load the CSV file at `dbfs:/FileStore/tmp/nl/treecover.csv` into a DataFrame.
- Use Apache Spark to read in the data, assigned to the variable `treeCoverDF`.
- Your method to get the CSV file into Databricks isn't graded. We are only concerned with how you use Spark to parse and load the actual data. 
- Please use the `inferSchema` option.

In [0]:
# YOUR CODE HERE
treeCoverDF = spark.read.format("csv").option("header","true").option("infer_schema","true").load("/FileStore/tmp/treecover.csv")
treeCoverDF.show(2)

+---+---------+------+-----+--------------------------------+------------------------------+-------------------------------+----------------------------------+----------+---------+---------------+---------+
| Id|Elevation|Aspect|Slope|Horizontal_Distance_To_Hydrology|Vertical_Distance_To_Hydrology|Horizontal_Distance_To_Roadways|Horizontal_Distance_To_Fire_Points|Cover_Type|Soil_Type|Wilderness_Area|Hillshade|
+---+---------+------+-----+--------------------------------+------------------------------+-------------------------------+----------------------------------+----------+---------+---------------+---------+
|  1|     2596|    51|    3|                             258|                             0|                            510|                              6279|         5|       29|              1|      9am|
|  2|     2590|    56|    2|                             212|                            -6|                            390|                              6225|         5|  

### Question 2:  Code Challenge - Print the Schema

Use Apache Spark to display the Schema of the `treeCoverDF` Dataframe.

In [0]:
treeCoverDF.printSchema()

root
 |-- Id: string (nullable = true)
 |-- Elevation: string (nullable = true)
 |-- Aspect: string (nullable = true)
 |-- Slope: string (nullable = true)
 |-- Horizontal_Distance_To_Hydrology: string (nullable = true)
 |-- Vertical_Distance_To_Hydrology: string (nullable = true)
 |-- Horizontal_Distance_To_Roadways: string (nullable = true)
 |-- Horizontal_Distance_To_Fire_Points: string (nullable = true)
 |-- Cover_Type: string (nullable = true)
 |-- Soil_Type: string (nullable = true)
 |-- Wilderness_Area: string (nullable = true)
 |-- Hillshade: string (nullable = true)



### Question 3:  Code Challenge - Rows & Columns

Use Apache Spark to display the number of rows and columns in the DataFrame.

In [0]:
print('No. of Rows :',treeCoverDF.count())
print('No. of Columns :',len(treeCoverDF.columns))

No. of Rows : 15120
No. of Columns : 12


#Part 2: Analysis

### Question 4:  Code Challenge - Summary Statistics for a Feature

Use Apache Spark to answer these questions about the `treeCoverDF` DataFrame:
- What is the range - minimum and maximum - of values for the feature `elevation`?
- What are the mean and standard deviation of the feature `elevation`?

In [0]:
from pyspark.sql.functions import col, asc, mean, variance

#treeCoverDF.agg({"elevation": 'min'}).show()
selectminDF = treeCoverDF.select("elevation").sort(treeCoverDF["elevation"])
print(selectminDF.collect()[0])

selectmaxDF = treeCoverDF.select("elevation").sort(treeCoverDF["elevation"].desc())
print(selectmaxDF.collect()[0])

meanDF = treeCoverDF.select(mean("elevation"))
print(meanDF.collect())

varianceDF = treeCoverDF.select(variance("elevation"))
print(varianceDF.collect())

Row(elevation='1863')
Row(elevation='3849')
[Row(avg(elevation)=2749.3225529100528)]
[Row(var_samp(elevation)=174455.0681863526)]


### Answer #4:

- Min `elevation`: `YOUR ANSWER HERE` : 1863
- Max `elevation`: `YOUR ANSWER HERE` : 3849
- Mean `elevation`: `YOUR ANSWER HERE` : 2749.3225529100528
- Standard Deviation of `elevation`: `YOUR ANSWER HERE` : 174455.0681863526

### Question 5:  Code Challenge - Record Count

Use Apache Spark to answer the following question:
- How many entries in the dataset have an `elevation` greater than or equal to 2749.32 meters **AND** a `Cover_Type` of 1 or 2?

In [0]:
from pyspark.sql.functions import col, asc
print(treeCoverDF.select("elevation").where((col("elevation") >= "2749.32") & ((col("Cover_Type") == "1") | (col("Cover_Type") == "2"))).count())

3883


### Question 6: Code Challenge - Compute a Percentage

Use Apache Spark to answer the following question:
- What percentage of entries with `Cover_Type` 1 or 2 have an `elevation` at or above 2749.32 meters?

In [0]:
from pyspark.sql.functions import *
elevation_df = treeCoverDF.select("elevation").where((col("Cover_Type") == "1") | (col("Cover_Type") == "2"))
percentage_df= elevation_df.select([(count(when((col("elevation") >= "2749.32"),col("elevation")))/count(lit(1))).alias(c) for c in elevation_df.columns])
percentage_df.show()

+------------------+
|         elevation|
+------------------+
|0.8988425925925926|
+------------------+



### Question 7: Code Challenge - Visualize Feature Distribution

Use any [visualization tool available in the Databricks Runtime](https://docs.databricks.com/user-guide/visualizations/index.html) to generate the following visualization:

- a bar chart that helps visualize the distribution of different Wilderness Areas in our dataset

In [0]:
from pyspark.sql.types import *

treeCoverDF = treeCoverDF.withColumn("elevation", treeCoverDF["elevation"].cast(IntegerType()))
treeCoverDF = treeCoverDF.withColumn("Wilderness_Area", treeCoverDF["Wilderness_Area"].cast(IntegerType()))
treeCoverDF = treeCoverDF.withColumn("Cover_Type", treeCoverDF["Cover_Type"].cast(IntegerType()))

treeCoverDF.registerTempTable("tree_cover_DF")
display(sqlContext.sql("select id, Wilderness_Area from tree_cover_DF"))

id,Wilderness_Area
1,1
2,1
3,1
4,1
5,1
6,1
7,1
8,1
9,1
10,1


### Question 8: Code Challenge - Visualize Average Elevation by Cover Type 

Use any [visualization tool available in the Databricks Runtime](https://docs.databricks.com/user-guide/visualizations/index.html) to generate the following visualization:

- a bar chart showing the average elevation of each cover type with string labels for cover type

**NOTE: you will need to match the integer values in the column `treeCoverDF.Cover_Type` to the string values in `dbfs:/FileStore/tmp/nl/covertype.csv` to retrieve the Cover Type Labels. It is recommended to use an Apache Spark join.**

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import IntegerType

coverTypeDF = spark.read.format("csv").option("header","true").option("infer_schema","true").load("/FileStore/tmp/covertype.csv")
coverTypeDF = coverTypeDF.withColumn("cover_type_key", coverTypeDF["cover_type_key"].cast(IntegerType()))

df_new=treeCoverDF.join(coverTypeDF,treeCoverDF.Cover_Type == coverTypeDF.cover_type_key, "inner")

display(df_new)

Id,elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Horizontal_Distance_To_Fire_Points,Cover_Type,Soil_Type,Wilderness_Area,Hillshade,cover_type_key,cover_type_label
1,2596,51,3,258,0,510,6279,5,29,1,9am,5,Aspen
2,2590,56,2,212,-6,390,6225,5,29,1,Noon,5,Aspen
3,2804,139,9,268,65,3180,6121,2,12,1,3pm,2,Lodgepole Pine
4,2785,155,18,242,118,3090,6211,2,30,1,9am,2,Lodgepole Pine
5,2595,45,2,153,-1,391,6172,5,29,1,Noon,5,Aspen
6,2579,132,6,300,-15,67,6031,2,29,1,3pm,2,Lodgepole Pine
7,2606,45,7,270,5,633,6256,5,29,1,9am,5,Aspen
8,2605,49,4,234,7,573,6228,5,29,1,Noon,5,Aspen
9,2617,45,9,240,56,666,6244,5,29,1,3pm,5,Aspen
10,2612,59,10,247,11,636,6230,5,29,1,9am,5,Aspen


#Part 3: Data Ingestion, Cleansing, and Transformations

## Instructions 

This is a multi-step, data pipeline question in which you need to achieve a few objectives to build a successful job.

### Data Sets

#### `u.data.csv`

- The full u data set, 100000 ratings by 943 users on 1682 items. 
- Each user has rated at least 20 movies.  
- Users and items are numbered consecutively from 1. 
- The data is randomly ordered. 
- This is a tab separated file consisting of four columns: 
   - user id 
   - movie id 
   - rating 
   - date (unix seconds since 1/1/1970 UTC)

#### Desired schema

- `user_id INTEGER`
- `movie_id INTEGER`
- `rating INTEGER`
- `date DATE `

#### `u.item.csv`

- This is a `|` separated file consisting of six columns:
   - movie id
   - movie title
   - release date
   - video release date
   - IMDb URL
   - genre
- movie ids in this file match movie ids in `u.data`.

#### Desired schema

- `movie_id INTEGER`
- `movie_title STRING`

### Question 9:  Code Challenge - Load DataFrames

Use Apache Spark to perform the following:
1. define the correct schemas for each Data Set to be imported as described above  
   **note:** 
      - for `u.data.csv`, `date` *must* be stored using `DateType` with the format `yyyy-MM-dd`
      - you may need to ingest `timestamp` data using `IntegerType`
      - be sure to drop unneccesary columns for `u.item.csv`
1. import the two files as DataFrames names `uDataDF` and `uItemDF` using the schemas you defined and these paths:
   - `dbfs:/FileStore/tmp/u.data.csv`
   - `dbfs:/FileStore/tmp/u.item.csv`
1. order the `uDataDF` DataFrame by the `date` column

**NOTE:** Please display the DataFrames, `uDataDF` and `uItemDF` after loading.

#### `uDataDF`

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
ddf = spark.read.option("delimiter","\t").format("csv").option("header","false").option("infer_schema","true").load("/FileStore/tmp/u.data.csv")
data_schema = [StructField('user_id',StringType(),True),StructField('movie_id',StringType(),True),StructField('rating',StringType(),True),StructField('date',StringType(),True)]
schema_data = StructType(fields = data_schema)
ddf.cache()
uDatadf = spark.createDataFrame(ddf.rdd,schema=schema_data)
uDatadf = uDatadf.withColumn("user_id", uDatadf["user_id"].cast(IntegerType()))
uDatadf = uDatadf.withColumn("movie_id", uDatadf["movie_id"].cast(IntegerType()))
uDatadf = uDatadf.withColumn("rating", uDatadf["rating"].cast(IntegerType()))
uDatadf = uDatadf.withColumn("date", to_date(from_unixtime(uDatadf["date"], "yyyy-MM-dd").cast(DateType())))
uDataDF = uDatadf.orderBy(col("date"))
uDataDF.show()

+-------+--------+------+----------+
|user_id|movie_id|rating|      date|
+-------+--------+------+----------+
|    119|     382|     5|1997-09-20|
|    119|     237|     5|1997-09-20|
|    119|     718|     5|1997-09-20|
|    276|      40|     3|1997-09-20|
|    276|     796|     1|1997-09-20|
|    276|      70|     4|1997-09-20|
|    119|    1153|     5|1997-09-20|
|    276|     417|     4|1997-09-20|
|    119|     222|     5|1997-09-20|
|    276|     188|     4|1997-09-20|
|    259|     255|     4|1997-09-20|
|    276|      64|     5|1997-09-20|
|    276|     142|     3|1997-09-20|
|    276|     181|     5|1997-09-20|
|     23|     323|     2|1997-09-20|
|    276|     294|     4|1997-09-20|
|     23|     196|     2|1997-09-20|
|    276|     324|     4|1997-09-20|
|    276|     564|     3|1997-09-20|
|    276|     419|     5|1997-09-20|
+-------+--------+------+----------+
only showing top 20 rows



#### `uItemDF`

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
ddf=spark.read.option("delimiter","|").format("csv").option("header","false").option("infer_schema","true").load("/FileStore/tmp/u.item.csv")
cols=["_c2","_c3","_c4","_c5","_c6","_c7","_c8","_c9","_c10","_c11","_c12","_c13","_c14","_c15","_c16","_c17","_c18","_c19","_c20","_c21","_c22","_c23"]
ddf1 = ddf.drop(*cols)
data_schema = [StructField('movie_id',StringType(),True),StructField('movie_title',StringType(),True)]
schema_data = StructType(fields = data_schema)
uItemDF = spark.createDataFrame(ddf1.rdd,schema=schema_data)
uItemDF = uItemDF.withColumn("movie_id", uItemDF["movie_id"].cast(IntegerType()))
uItemDF.show()

+--------+--------------------+
|movie_id|         movie_title|
+--------+--------------------+
|       1|    Toy Story (1995)|
|       2|    GoldenEye (1995)|
|       3|   Four Rooms (1995)|
|       4|   Get Shorty (1995)|
|       5|      Copycat (1995)|
|       6|Shanghai Triad (Y...|
|       7|Twelve Monkeys (1...|
|       8|         Babe (1995)|
|       9|Dead Man Walking ...|
|      10|  Richard III (1995)|
|      11|Seven (Se7en) (1995)|
|      12|Usual Suspects, T...|
|      13|Mighty Aphrodite ...|
|      14|  Postino, Il (1994)|
|      15|Mr. Holland's Opu...|
|      16|French Twist (Gaz...|
|      17|From Dusk Till Da...|
|      18|White Balloon, Th...|
|      19|Antonia's Line (1...|
|      20|Angels and Insect...|
+--------+--------------------+
only showing top 20 rows



### Question 10:  Code Challenge - Perform a Join

Use Apache Spark to do the following:
- join `uDataDF` and `uItemDf` on `movie_id` as a new DataFrame called `uMovieDF`  
   **note:** make sure you do not create duplicate `movie_id` columns
   
**NOTE:** Please display the DataFrame `uMovieDF`.

In [0]:
uMovieDF = uDataDF.join(uItemDF,uDataDF.movie_id == uItemDF.movie_id,"inner").select(uDataDF.user_id, uDataDF.movie_id, uDataDF.rating,uDataDF.date,uItemDF.movie_title)
uMovieDF.show()

+-------+--------+------+----------+--------------------+
|user_id|movie_id|rating|      date|         movie_title|
+-------+--------+------+----------+--------------------+
|    250|     496|     4|1997-10-29|It's a Wonderful ...|
|    293|     471|     3|1998-03-03|Courage Under Fir...|
|    305|     471|     4|1998-02-01|Courage Under Fir...|
|    203|     471|     4|1997-11-25|Courage Under Fir...|
|      6|     463|     4|1997-12-31|Secret of Roan In...|
|    124|     496|     1|1998-03-19|It's a Wonderful ...|
|    186|     148|     4|1998-04-04|Ghost and the Dar...|
|    201|     148|     1|1998-01-07|Ghost and the Dar...|
|     23|     463|     4|1997-09-20|Secret of Roan In...|
|     94|     471|     4|1998-04-04|Courage Under Fir...|
|    222|     148|     2|1997-12-02|Ghost and the Dar...|
|    265|     471|     4|1997-09-27|Courage Under Fir...|
|    101|     471|     3|1997-10-18|Courage Under Fir...|
|    251|     148|     2|1998-01-31|Ghost and the Dar...|
|     59|     

### Question 11:  Code Challenge - Perform an Aggregation

Use Apache Spark to do the following:
1. create an aggregate DataFrame, `aggDF` by
  1. extracting the year from the `date` (of the review)
  1. getting the average rating of each film per year as a column named `average_rating`
  1. ordering descending by year and average rating
1. write the resulting dataframe to a table named "movie_by_year_average_rating" in the Default database  
   **note:** use `mode(overwrite)` 

#### Desired Schema
The schema of you resulting DataFrame should be:
- `year INTEGER`
- `movie_title STRING`
- `average_rating DOUBLE`

**NOTE:** Please display the DataFrame `aggDF`.

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
affDF_1 = uMovieDF.withColumn("year",year(uMovieDF["date"]))
affDF_2 = affDF_1.groupBy("movie_title","year").avg("rating")
affDF_3 = affDF_2.withColumn("average_rating",affDF_2["avg(rating)"]).drop("avg(rating)")
affDF = affDF_3.select("year","movie_title","average_rating").orderBy(desc("year"),desc("average_rating"))
affDF.show()
affDF.write.mode("overwrite").saveAsTable("movie_by_year_average_rating")

+----+--------------------+-----------------+
|year|         movie_title|   average_rating|
+----+--------------------+-----------------+
|1998|Visitors, The (Vi...|              5.0|
|1998|The Deadly Cure (...|              5.0|
|1998|Great Day in Harl...|              5.0|
|1998|     Star Kid (1997)|              5.0|
|1998|Marlene Dietrich:...|              5.0|
|1998|Entertaining Ange...|              5.0|
|1998|Mina Tannenbaum (...|              5.0|
|1998|        Faust (1994)|              5.0|
|1998|  Prefontaine (1997)|              5.0|
|1998|Year of the Horse...|             4.75|
|1998|Pather Panchali (...|4.666666666666667|
|1998|         Safe (1995)|              4.6|
|1998|      Everest (1998)|              4.5|
|1998|     Faithful (1996)|              4.5|
|1998|Some Mother's Son...|              4.5|
|1998|    8 Seconds (1994)|              4.5|
|1998|Grateful Dead (1995)|              4.5|
|1998|Hotel de Love (1996)|              4.5|
|1998|Wallace & Gromit:...|       

## Part 4: Fun with JSON

JSON values are typically passed by message brokers such as Kafka or Kinesis in a string encoding. When consumed by a Spark Structured Streaming application, this json must be converted into a nested object in order to be used.

Below is a list of json strings that represents how data might be passed from a message broker.

**Note:** Make sure to run the cell below to retrieve the sample data.

In [0]:
%python


sampleJson = [
 ('{"user":100, "ips" : ["191.168.192.101", "191.168.192.103", "191.168.192.96", "191.168.192.99"]}',), 
 ('{"user":101, "ips" : ["191.168.192.102", "191.168.192.105", "191.168.192.103", "191.168.192.107"]}',), 
 ('{"user":102, "ips" : ["191.168.192.105", "191.168.192.101", "191.168.192.105", "191.168.192.107"]}',), 
 ('{"user":103, "ips" : ["191.168.192.96", "191.168.192.100", "191.168.192.107", "191.168.192.101"]}',), 
 ('{"user":104, "ips" : ["191.168.192.99", "191.168.192.99", "191.168.192.102", "191.168.192.99"]}',), 
 ('{"user":105, "ips" : ["191.168.192.99", "191.168.192.99", "191.168.192.100", "191.168.192.96"]}',), 
]

### Question 12:  Code Challenge - Count the IPs

Use any coding techniques known to you to parse this list of JSON strings to answer the following question:
- how many occurrences of each IP address are in this list?

#### Desired Output
Your results should be this:


| ip | count |
|:-:|:-:|
| `191.168.192.96` | `3` |
| `191.168.192.99` | `6` |
| `191.168.192.100` | `2` |
| `191.168.192.101` | `3` |
| `191.168.192.102` | `2` |
| `191.168.192.103` | `2` |
| `191.168.192.105` | `3` |
| `191.168.192.107` | `3` |

**NOTE:** The order of your results is not important.

In [0]:
import ast
from pyspark.sql.functions import *
rdd_1 = spark.sparkContext.parallelize(sampleJson)
rdd_2 = rdd_1.map(lambda x:ast.literal_eval(x[0]))
df = spark.read.json(rdd_2)
count_the_ips_df = df.select(explode("ips").alias("ip"))
count_the_ips_df.groupby(col("ip")).count().show()

+---------------+-----+
|             ip|count|
+---------------+-----+
| 191.168.192.96|    3|
|191.168.192.101|    3|
|191.168.192.103|    2|
| 191.168.192.99|    6|
|191.168.192.105|    3|
|191.168.192.107|    3|
|191.168.192.102|    2|
|191.168.192.100|    2|
+---------------+-----+



## Part 5: The Databricks API

### Question 13: Conceptual Question - the Databricks API

In 4-5 sentences, please explain what the Databricks API is used for at a high-level.

### Answer:

It allows for programmatic management of various Databricks resources easily.

Key Benifits :
Speed
Real-time Stream Processing
Supports Multiple Workloads
Increased Usability
Advanced Analytics

Key Features of REST API:  FLEXIBLE, LAYERED, STATELESS

### Question 14: Conceptual Question - Explain an API Call

In 4-5 sentences, please explain what this API call. Be sure to discuss some key attributes about the cluster.

```
$ curl -n -X POST -H 'Content-Type: application/json'                      \
  -d '{                                                                     \
  "cluster_name": "high-concurrency-cluster",                               \
  "spark_version": "4.2.x-scala2.11",                                       \
  "node_type_id": "i3.xlarge",                                              \
  "spark_conf":{                                                            \
        "spark.databricks.cluster.profile":"serverless",                    \
        "spark.databricks.repl.allowedLanguages":"sql,python,r"             \
     },                                                                     \
     "aws_attributes":{                                                     \
        "zone_id":"us-west-2c",                                             \
        "first_on_demand":1,                                                \
        "availability":"SPOT_WITH_FALLBACK",                                \
        "spot_bid_price_percent":100                                        \
     },                                                                     \
   "custom_tags":{                                                          \
        "ResourceClass":"Serverless"                                        \
     },                                                                     \
       "autoscale":{                                                        \
        "min_workers":1,                                                    \
        "max_workers":2                                                     \
     },                                                                     \
  "autotermination_minutes":10                                              \
}' https://dogfood.staging.cloud.databricks.com/api/2.0/clusters/create '
```

### Answer:

`EDIT THIS MARKDOWN CELL WITH YOUR REPLY`

## Part 6: Security

### Question 15: Conceptual Question - Security on Databricks

Using the Databricks Documentation, what would you recommend to a Databricks and AWS customer for **securely** storing and accessing their data.

### Answer:

`EDIT THIS MARKDOWN CELL WITH YOUR REPLY`

# This is the end of the official test. Bonus below!

## Part 7: Bonus: Data Science & Machine Learning

### Question 16: Conceptual Question - A Skewed Feature

One of these lines is the *mean* of this feature. The other is the *median*. Which of these lines is the **mean** - the red line or the black line?

<img width=400px src=https://www.evernote.com/l/AAEycL6CQ0hLi5V5pIo91Ko-Pfk2i0AnGyMB/image.png>

### Answer:

Black Line

### Question 17: Conceptual Question - Exploratory Data Analysis

The plots below show the distribution of home selling prices differentiated by a few categorical features. Based on these plots, **which of these categorical features** - Property Type, Exterior Quality, or Month Sold - would you expect to be most associated with Price? **Why**?

<img width=1600px src=https://www.evernote.com/l/AAHulkcc20hHSJV6D1udKiwSDCN0S6oV_5YB/image.png>

### Answer:

Exterior Quality beacause it clearly showing us the impact of different quality categories on Sales price and dnot have confusion like other two graphs.

### Question 18: Conceptual Question - Analyze Model Performance

Consider the following results for a decision tree model against training and testing data sets:

`decision tree regression - train r2 score: 0.9944`  
`decision tree regression - test r2 score:  0.3119`


What is your assessment of this model?

### Answer:

A higher value of R2 is desirable as it indicates better results. so for training dataset we are geeting better accurate result but for test data set effciency is less.

### Question 19: Conceptual Question - Model Selection

A series of models has been built using the same training data, but each with a subset of features.

Consider the following results for a series of logistic regression models and then answer this question:

- Which model would you choose and why?
- What other things would you want to look at and why?

| model number | feature subset | logistic regression test accuracy|
|:-:|:-:|:-:|
| 1| feat_1 |	 0.631|
| 2| feat_2 |	 0.552|
| 3| feat_3 |	 0.868|
| 4| feat_4 |	 0.868|
| 5| feat_1, feat_2 |	 0.657|
| 6| feat_1, feat_3 |	 0.947|
| 7| feat_1, feat_4 |	 0.921|
| 8| feat_2, feat_3 |	 0.947|
| 9| feat_2, feat_4 |	 0.973|
| 10| feat_3, feat_4 |	 0.947|
| 11| feat_1, feat_2, feat_3 |	 0.947|
| 12| feat_1, feat_2, feat_4 |	 0.947|
| 13| feat_1, feat_3, feat_4 |	 0.947|
| 14| feat_2, feat_3, feat_4 |	 0.973|
| 15| feat_1, feat_2, feat_3, feat_4 |	 0.973|

### Answer:

`EDIT THIS MARKDOWN CELL WITH YOUR REPLY`

-sandbox
&copy; 2019 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>